In [1]:
import json
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
import os
os.chdir("C:/Users/Sankar/SDN_Project/JSON/JSON")
files=os.listdir('C:/Users/Sankar/SDN_Project/JSON/JSON/')
data=[]
for file in files:    
    with open(file) as f:
        df = json.load(f)
        data.append(df)

In [3]:
def getSourceDestination(data):
    if 'dnsname' in str(data):
        src_net_key = 'ietf-acldns:src-dnsname'
        dest_net_key = 'ietf-acldns:dst-dnsname'
    elif 'network' in str(data):
        src_net_key = 'source-ipv4-network'
        dest_net_key = 'destination-ipv4-network'
    else:
        src_net_key = '*'
        dest_net_key = '*'
    return data.get(src_net_key, '*'), data.get(dest_net_key, '*'), data.get('protocol', '*')
    

In [4]:
final_list = []
for i in range(len(data)):
    access_json=data[i].get('ietf-access-control-list:access-lists').get('acl')    
    for acl in access_json:
        aces = acl['aces']['ace']
        for ace in aces:
            final_row = {
            'sEth': '*',
            'dEth': '*',
            'typEth': '*',
            }
            if 'ethertype' in str(ace['matches']):
                final_row['sEth'] = ace['matches']['eth'].get('source-mac-address', '*')
                final_row['dEth'] = ace['matches']['eth'].get('destination-mac-address', '*')
                final_row['typEth'] = ace['matches']['eth'].get('ethertype', '*')
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            elif 'ipv6' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv6'])
                final_row['Source'] = source
                final_row['Destination'] = dest
                final_row['proto'] = proto
            else:
                final_row['Source'] = '*'
                final_row['Destination'] = '*'
                final_row['proto'] = '*'
            if 'ipv4' in str(ace['matches']):
                source, dest, proto = getSourceDestination(ace['matches']['ipv4'])
                if proto in [1,6,17] :
                    final_row['typEth']='0x0800'
            if 'icmp' in str(ace['matches']):
                final_row['type'] = ace['matches']['icmp'].get('type', '*')
                final_row['code'] = ace['matches']['icmp'].get('code', '*')
            else:
                final_row['type'] = '*' 
                final_row['code'] = '*'   
            if 'source-port' in str(ace['matches']):
                final_row['sPort'] = ace['matches']['udp']['source-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['source-port'].get('port', '*')
            else:
                final_row['sPort'] = '*'
            if 'destination-port' in str(ace['matches']):
                final_row['dPort'] = ace['matches']['udp']['destination-port'].get('port', '*') if 'udp' in str(ace['matches']) else ace['matches']['tcp']['destination-port'].get('port', '*')
            else:
                final_row['dPort'] = '*'
            final_row['priority'] = '*'
            final_row['action'] = 'forward' if ace['actions'].get('forwarding', '') == 'accept' else '*'
            if final_row['Source'] == '*' and final_row['Destination'] != '*':
                final_row['sEth'] = '9e:8d:de:80:29:28'  
            if final_row['Destination'] == '*' and final_row['Source'] != '*':
                final_row['dEth'] = '9e:8d:de:80:29:28'     
            final_list.append(final_row)

df = pd.DataFrame(final_list)
df


,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
0,9e:8d:de:80:29:28,*,0x0800,*,dcape-na.amazon.com,6,*,*,*,443,*,forward
1,9e:8d:de:80:29:28,*,0x0800,*,softwareupdates.amazon.com,6,*,*,*,443,*,forward
2,9e:8d:de:80:29:28,*,0x0800,*,3.north-america.pool.ntp.org,17,*,*,*,123,*,forward
3,9e:8d:de:80:29:28,*,*,*,224.0.0.22/32,2,*,*,*,*,*,forward
4,9e:8d:de:80:29:28,*,0x0800,*,239.255.255.250/32,17,*,*,*,1900,*,forward
5,9e:8d:de:80:29:28,ff:ff:ff:ff:ff:ff,0x0800,*,255.255.255.255/32,17,*,*,*,67,*,forward
6,9e:8d:de:80:29:28,*,0x0800,*,ntp-g7g.amazon.com,17,*,*,*,123,*,forward
7,9e:8d:de:80:29:28,*,0x0800,*,todo-ta-g7g.amazon.com,6,*,*,*,443,*,forward
8,*,*,0x0800,*,*,1,8,0,*,*,*,forward
9,9e:8d:de:80:29:28,*,0x0800,*,www.example.org,6,*,*,*,80,*,forward


In [5]:
df.loc[len(df.index)] = ['*','*','*','*','*','*','*','*','*','*','*','deny']

In [6]:
df.tail()

,sEth,dEth,typEth,Source,Destination,proto,type,code,sPort,dPort,priority,action
1021,*,9e:8d:de:80:29:28,0x0800,scalews.withings.net,*,6,*,*,443,*,*,forward
1022,*,*,0x0800,*,*,17,*,*,67,*,*,forward
1023,*,9e:8d:de:80:29:28,0x0800,scalews.withings.net,*,6,*,*,80,*,*,forward
1024,*,*,0x888e,*,*,*,*,*,*,*,*,forward
1025,*,*,*,*,*,*,*,*,*,*,*,deny


In [7]:
df.to_csv('C:/Users/Sankar/SDN_Project/ACL_All_IOT_Devices.csv')

In [8]:
column = ['sEth','dEth','typEth','Source','Destination','proto','sPort','dPort','priority','action']

In [9]:
df2 = df[column]
df1 =df2.drop_duplicates()

In [10]:
df1.shape

(698, 10)

In [11]:
df1.to_csv('C:/Users/Sankar/SDN_Project/Project_data_final.csv')

In [12]:
#raw_df = pd.read_excel("C:/Users/Sankar/SDN_Project/Project_data_final.xlsx")

In [13]:
#type(df1)

In [14]:
#raw_df=raw_df.drop('priority', axis=1)
#raw_df.to_csv('C:/Users/Sankar/SDN_Project/Project_data_final_converted.csv',index=None,header=True)

In [15]:
##import csv 
##csv_file = 'C:/Users/Sankar/SDN_Project/Project_data_final_converted.csv' 
##txt_file = 'C:/Users/Sankar/SDN_Project/Project_data_final_text.txt'
##with open(txt_file, "w") as my_output_file:
##   with open(csv_file, "r") as my_input_file:
##        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
##    my_output_file.close()*/